In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

# Verify Spark session creation
spark.sql("SHOW DATABASES").show()

25/04/27 11:38:10 WARN Utils: Your hostname, codespaces-785606 resolves to a loopback address: 127.0.0.1; using 10.0.1.252 instead (on interface eth0)
25/04/27 11:38:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/workspaces/learn_iceberg/iceberg_pyspark_project/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f1453ee5-9af2-464c-8511-67e70c01de68;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
:: resolution report :: resolve 353ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apa

+---------+
|namespace|
+---------+
|  default|
+---------+



25/04/27 11:38:33 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
# Create an Iceberg table
spark.sql("""
  CREATE TABLE local.schema.customers_ice_concepts_cow (
    id INT,
    name STRING,
    age INT
  )
  USING iceberg
  TBLPROPERTIES (
    'write.delete.mode' = 'copy-on-write',
    'write.update.mode' = 'copy-on-write',
    'write.merge.mode' = 'copy-on-write'
  )
""")

DataFrame[]

In [3]:
# Insert some sample data
spark.sql("""
  INSERT INTO local.schema.customers_ice_concepts_cow VALUES
    (1, 'Alice', 30),
    (2, 'Bob', 25),
    (3, 'Charlie', 35)""")

DataFrame[]

In [9]:
# Query the data
result = spark.sql("SELECT * FROM local.schema.customers_ice_concepts_cow")
result.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  3|Mohan| 35|
|  1|Alice| 30|
+---+-----+---+



In [ ]:
spark.sql("SELECT * FROM local.schema.customers_ice_concepts where name = 'Alice'").show()

In [6]:
spark.sql("""delete from local.schema.customers_ice_concepts_cow  WHERE id = 2""")


DataFrame[]

In [8]:
spark.sql("""
  UPDATE local.schema.customers_ice_concepts_cow
  SET name = 'Mohan'
  WHERE id = 3
""")

DataFrame[]

MERGE ON READ

In [2]:
# Create an Iceberg table
spark.sql("""
  CREATE TABLE local.schema.customers_ice_concepts_mor (
    id INT,
    name STRING,
    age INT
  )
  USING iceberg
  TBLPROPERTIES (
    'write.delete.mode' = 'merge-on-read',
    'write.update.mode' = 'merge-on-read',
    'write.merge.mode' = 'merge-on-read'
  )
""")

DataFrame[]

In [3]:
# Insert some sample data
spark.sql("""
  INSERT INTO local.schema.customers_ice_concepts_mor VALUES
    (1, 'Alice', 30),
    (2, 'Bob', 25),
    (3, 'Charlie', 35)""")

DataFrame[]

In [7]:
# Query the data
result = spark.sql("SELECT * FROM local.schema.customers_ice_concepts_mor")
result.show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|Alice| 30|
|  3|Mohan| 35|
+---+-----+---+



In [5]:
spark.sql("""delete from local.schema.customers_ice_concepts_mor  WHERE id = 2""")


DataFrame[]

In [6]:
spark.sql("""
  UPDATE local.schema.customers_ice_concepts_mor
  SET name = 'Mohan'
  WHERE id = 3
""")

DataFrame[]